In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

from keras import Sequential
from keras.layers import Dense, Flatten

2024-09-11 18:32:25.656514: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-11 18:32:25.666634: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 18:32:25.677921: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 18:32:25.681475: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-11 18:32:25.690518: I tensorflow/core/platform/cpu_feature_guar

In [4]:
axes = ['relativeAltitude', 'gravityX', 'gravityY', 'gravityZ', 'rotationRateX', 'rotationRateY', 'rotationRateZ']
#axes = ['gravityX', 'gravityY', 'gravityZ', 'rotationRateX', 'rotationRateY', 'rotationRateZ']
window_size=500
total_n = 2000

In [14]:
u = pd.read_json("deadlift.json")

#run through filters


In [15]:
u[axes[0]]

0       0.00
1       0.01
2      -0.01
3      -0.04
4      -0.06
        ... 
1584     NaN
1585     NaN
1586     NaN
1587     NaN
1588     NaN
Name: relativeAltitude, Length: 1589, dtype: float64

In [16]:
u[axes[1]]

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
1584    0.911575
1585    0.911579
1586    0.911732
1587    0.911952
1588    0.912168
Name: gravityX, Length: 1589, dtype: float64

In [17]:
u = pd.read_json("bench.json")

scaler = Normalizer()
a = u[axes[0]].dropna().to_numpy().reshape(-1, 1)
scaler.fit_transform(a)

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [18]:
a

array([[0.10000002],
       [0.07999992],
       [0.04999995],
       [0.07999992],
       [0.13      ],
       [0.16999996],
       [0.18999994]])

In [19]:
u[axes[0]][:10]

0    0.10
1    0.08
2    0.05
3    0.08
4    0.13
5    0.17
6    0.19
7     NaN
8     NaN
9     NaN
Name: relativeAltitude, dtype: float64

In [11]:
def parse_json(path, axes=axes):
    u = pd.read_json(path)
    out = []
    #scaler = StandardScaler()
    for i in tqdm(range(0,total_n - window_size), desc=path):
        temp = []
        for a in axes:
            if np.nan in u[a][i:i+500]:
                continue
            a = np.array([e for e in u[a][i:i+500] if e is not np.nan]).reshape(-1, 1)
            temp.append(scaler.fit_transform(a).squeeze())

        out.append(np.array(temp))
    
    return out
deadlift = parse_json('deadlift.json')
bench = parse_json('bench.json')

deadlift.json:   0%|                                   | 0/1500 [00:00<?, ?it/s]/home/neal/robin_yo_hood/.quant/lib/python3.12/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/neal/robin_yo_hood/.quant/lib/python3.12/site-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/home/neal/robin_yo_hood/.quant/lib/python3.12/site-packages/sklearn/utils/extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/home/neal/robin_yo_hood/.quant/lib/python3.12/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/neal/robin_yo_hood/.quant/lib/python3.12/site-packages/sklearn/utils/extmath.py:1142: RuntimeWarning: invalid value enc

KeyboardInterrupt: 

In [8]:
model = Sequential()
model.add(Flatten())

# model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(6, 500)))
# model.add(MaxPooling1D(pool_size=2))

# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))


model.add(Dense(6*500, activation='relu'))

# Fully connected layers
model.add(Dense(900, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))


In [9]:
classes = {'deadlift': 1, 'bench': 0}

#x = np.array(deadlift[:1001]).concat(bench[:1000]) #np.concat(deadlift, bench)
x = np.concatenate((deadlift[:1000], bench[:1000]))
y = np.concatenate((np.array([classes['deadlift']] * 1000), np.array([classes['bench']] * 1000) ))


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.6, shuffle=True)



In [10]:
x_train[3].shape

(6, 500)

In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train)

2024-09-10 23:26:36.334339: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-09-10 23:26:36.334356: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:135] retrieving CUDA diagnostic information for host: neal-ASUSLaptop-Q530VJ
2024-09-10 23:26:36.334359: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:142] hostname: neal-ASUSLaptop-Q530VJ
2024-09-10 23:26:36.334484: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:166] libcuda reported version is: 535.183.1
2024-09-10 23:26:36.334494: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] kernel reported version is: 535.183.1
2024-09-10 23:26:36.334496: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:249] kernel version seems to match DSO: 535.183.1


25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - accuracy: 0.6515 - loss: 0.5465


In [12]:
y_pred = model.predict(x_test)


38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [13]:
y_pred = [0 if e < 0.5 else 1 for e in y_pred]

In [14]:
accuracy_score(y_test, y_pred)

0.495